In [21]:
from qdrant_client import QdrantClient
client = QdrantClient(url="http://localhost:6333")
client_grpc = QdrantClient(url="http://localhost:6334", prefer_grpc=True)

In [26]:
import numpy as np
from tqdm import tqdm

In [27]:
from qdrant_client.models import Distance, VectorParams, PointStruct

In [29]:
from qdrant_client.models import Filter, FieldCondition, Range, OptimizersConfigDiff
from qdrant_client.http.models import SearchRequest

# test indexing speed

## vanilla http

In [6]:
client.recreate_collection(
    collection_name="clip_image_product",
    vectors_config=VectorParams(size=512, distance=Distance.COSINE),
    # hnsw_config=OptimizersConfigDiff(
    #     indexing_threshold=int(1e8 - 1e3),
    # )
)
assert client.count(collection_name="clip_image_product").count == 0
id_start = 0
payloads = [{"color": "red", "rand_number": idx % 10} for idx in range(int(1e3))]


for i in tqdm(range(int(1e4))):
    ids = list(range(id_start, id_start + int(1e2)))
    vectors = np.random.rand(int(1e2), 512)
    client.upload_collection(
        collection_name="clip_image_product",
        vectors=vectors.tolist(),
        payload=payloads,
        ids=ids
    )
    id_start += int(1e2)
    if i > 50:
        break

  1%|          | 51/10000 [00:12<39:49,  4.16it/s]


## http with indexing off

In [7]:
client.recreate_collection(
    collection_name="clip_image_product",
    vectors_config=VectorParams(size=512, distance=Distance.COSINE),
    hnsw_config=OptimizersConfigDiff(
        indexing_threshold=int(1e6 - 1e3),
    )
)
assert client.count(collection_name="clip_image_product").count == 0
id_start = 0
payloads = [{"color": "red", "rand_number": idx % 10} for idx in range(int(1e3))]


for i in tqdm(range(int(1e4))):
    ids = list(range(id_start, id_start + int(1e2)))
    vectors = np.random.rand(int(1e2), 512)
    client.upload_collection(
        collection_name="clip_image_product",
        vectors=vectors.tolist(),
        payload=payloads,
        ids=ids
    )
    id_start += int(1e2)
    if i > 50:
        break

  1%|          | 51/10000 [00:12<40:18,  4.11it/s]


## vanilla grpc

In [10]:
client.recreate_collection(
    collection_name="clip_image_product",
    vectors_config=VectorParams(size=512, distance=Distance.COSINE),
    # hnsw_config=OptimizersConfigDiff(
    #     indexing_threshold=int(1e8 - 1e3),
    # )
)
assert client.count(collection_name="clip_image_product").count == 0
id_start = 0
payloads = [{"color": "red", "rand_number": idx % 10} for idx in range(int(1e3))]


for i in tqdm(range(int(1e4))):
    ids = list(range(id_start, id_start + int(1e2)))
    vectors = np.random.rand(int(1e2), 512)
    client_grpc.upload_collection(
        collection_name="clip_image_product",
        vectors=vectors,
        payload=payloads,
        ids=ids
    )
    id_start += int(1e2)
    if i > 50:
        break

  1%|          | 51/10000 [00:01<03:51, 43.00it/s]


## grpc with indexing off

In [11]:
client.recreate_collection(
    collection_name="clip_image_product",
    vectors_config=VectorParams(size=512, distance=Distance.COSINE),
    hnsw_config=OptimizersConfigDiff(
        indexing_threshold=int(1e6 - 1e3),
    )
)
assert client.count(collection_name="clip_image_product").count == 0
id_start = 0
payloads = [{"color": "red", "rand_number": idx % 10} for idx in range(int(1e3))]


for i in tqdm(range(int(1e4))):
    ids = list(range(id_start, id_start + int(1e2)))
    vectors = np.random.rand(int(1e2), 512)
    client_grpc.upload_collection(
        collection_name="clip_image_product",
        vectors=vectors,
        payload=payloads,
        ids=ids
    )
    id_start += int(1e2)
    if i > 50:
        break

  1%|          | 51/10000 [00:00<02:47, 59.23it/s]


# test search speed

## upload 1M data fast with vanilla grpc

In [12]:
client.recreate_collection(
    collection_name="clip_image_product",
    vectors_config=VectorParams(size=512, distance=Distance.COSINE)
)
assert client.count(collection_name="clip_image_product").count == 0
id_start = 0
payloads = [{"color": "red", "rand_number": idx % 10} for idx in range(int(1e3))]


for i in tqdm(range(int(1e4))):
    ids = list(range(id_start, id_start + int(1e2)))
    vectors = np.random.rand(int(1e2), 512)
    client_grpc.upload_collection(
        collection_name="clip_image_product",
        vectors=vectors,
        payload=payloads,
        ids=ids
    )
    id_start += int(1e2)

100%|██████████| 10000/10000 [03:35<00:00, 46.38it/s]


## search with HTTP no batch

In [13]:
for i in tqdm(range(100)):
    query_vector = np.random.rand(512)
    hits = client.search(
        collection_name="clip_image_product",
        query_vector=query_vector,
        limit=1000
    )

100%|██████████| 100/100 [00:03<00:00, 29.13it/s]


In [14]:
%%timeit
query_vector = np.random.rand(512)
hits = client.search(
    collection_name="clip_image_product",
    query_vector=query_vector,
    limit=1000
)

28.2 ms ± 570 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


## search with grpc no batch

In [15]:
for i in tqdm(range(100)):
    query_vector = np.random.rand(512)
    hits = client_grpc.search(
        collection_name="clip_image_product",
        query_vector=query_vector,
        limit=1000
    )

100%|██████████| 100/100 [00:02<00:00, 41.07it/s]


In [16]:
%%timeit
query_vector = np.random.rand(512)
hits = client_grpc.search(
    collection_name="clip_image_product",
    query_vector=query_vector,
    limit=1000
)

23.5 ms ± 252 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


## search with http batch

In [22]:
%%time
results = client.search_batch(
    collection_name="clip_image_product",
    requests=[
        SearchRequest(
            vector=np.random.rand(512).tolist(),
            limit=1000,
        ) for _ in range(100)
    ]
)

CPU times: user 1.44 s, sys: 124 ms, total: 1.57 s
Wall time: 2.22 s


## search with grpc batch

In [23]:
%%time
results = client_grpc.search_batch(
    collection_name="clip_image_product",
    requests=[
        SearchRequest(
            vector=np.random.rand(512).tolist(),
            limit=1000,
        ) for _ in range(100)
    ]
)

CPU times: user 1.01 s, sys: 75.8 ms, total: 1.09 s
Wall time: 1.81 s


## forget about http, grpc is faster, so let's focus on optimize grpc

In [34]:
import asyncio
client_grpc = QdrantClient(url="http://localhost:6334", prefer_grpc=True)

async def test_grpc_batch(requests):
    res = await client_grpc.search_batch(
        collection_name="clip_image_product",
        requests=requests
    )
    return res
requests=[[
    SearchRequest(
        vector=np.random.rand(512).tolist(),
        limit=1000,
    ) for _ in range(10)
] for __ in range(100)]
reses = []
for reqs in requests:
    reses.append(test_grpc_batch(reqs))
final_reses = await asyncio.gather(*reses)

TypeError: object list can't be used in 'await' expression